# Project: Investigate, Analysis, And Prediction For "Bank Loan" Dataset
### Data Link : https://www.kaggle.com/datasets/itsmesunil/bank-loan-modelling

## Table of Contents
<ul>
<li><a href="#dw">Data Wrangling</a></li>
<li><a href="#dc">Data Cleaing</a></li>
<li><a href="#dfl">Data Final Look</a></li>
<li><a href="#eda">Exploratory Data Analysis</a></li>
<li><a href="#ml">Random Forest Classifier Model</a></li>
<li><a href="#sum">Summary</a></li>
<li><a href="#con">Conclusions</a></li>
</ul>



# About The Data

This case is about a bank (Thera Bank) which has a growing customer base. Majority of these customers are liability customers (depositors) with varying size of deposits. The number of customers who are also borrowers (asset customers) is quite small, and the bank is interested in expanding this base rapidly to bring in more loan business and in the process, earn more through the interest on loans. In particular, the management wants to explore ways of converting its liability customers to personal loan customers (while retaining them as depositors). A campaign that the bank ran last year for liability customers showed a healthy conversion rate of over 9% success. This has encouraged the retail marketing department to devise campaigns to better target marketing to increase the success ratio with a minimal budget

# Dataset Focuse on:
* How to sell more Personal Loan products to Bank customers.
* How to know the better target client to increase the success ratio with a minimal budget.
* To identify the potential customers who have a higher probability of purchasing the loan.

## EDA Questions:
1- What Columns that affect taking personal loan ? <br>
2- How Strong do the columns affect it ? <br>

# Columns Description

**ID:** Customer ID						
**Age:** Customer's age in completed years						
**Experience:** years of professional experience						
**Income:** Annual income of the customer  					
**ZIPCode:** Home Address ZIP code					
**Family:** Family size of the customer						
**CCAvg:** Avg spending on credit cards per month  						
**Education:** Education Level. 1: Undergrad; 2: Graduate; 3: Advanced/Professional				
**Mortgage:** Value of house mortgage if any						
**Personal Loan:** Did this customer accept the personal loan offered in the last campaign?						
**Securities:** Account	Does the customer have a securities account with the bank?						
**CD Account:** Does the customer have a certificate of deposit (CD) account with the bank?						
**Online:** Does the customer use internet banking facilities?						
**CreditCard:** Does the customer use a credit card issued by UniversalBank?						

In [1]:
pip install openpyxl

In [2]:
#import used libs

import numpy as np
import pandas as pd
import seaborn as sn
import plotly.express as px
from plotly.offline import init_notebook_mode, iplot
init_notebook_mode(connected=True)
import matplotlib.pyplot as plt
from sklearn import metrics
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

In [3]:
# Read The Data
df = pd.read_excel('../input/bank-loan-modelling/Bank_Personal_Loan_Modelling.xlsx', sheet_name='Data')

df.head()

<a id='dw'></a>
# Data Wrangling


<img src="https://media4.giphy.com/media/wRKeX8o1eIxxu/giphy.gif?cid=ecf05e47oi023xu5rysigmm5k0f7i9uticu3t3qs8gtq7g1h&rid=giphy.gif&ct=g"/>

In [4]:
df.info()

In [5]:
df.describe().transpose()

In [6]:
plt.rcParams["figure.figsize"] = (20,10)
df.hist();

<h6>1- Here is Some Negative Values To Deal With It</h6> 

In [7]:
df.query('Experience < 0')

<h6>2- There is no duplicated rows</h6>

In [8]:
df.duplicated().value_counts()

<h6>3- Here is shown that there is no null values</h6>

In [9]:
df.isnull().sum()

<h6>4- Detecting outliers </h6>

In [10]:
px.box(df.drop(['ID','ZIP Code'],axis=1)).show()

<h5>[ Income , CCAvg , Mortgage ] columns Have Outliers</h5>

<h3>4.1- Check Outliers For Income Column</h3>

In [11]:
i='Income'

Q1 = df[i].quantile(0.25)
Q3 = df[i].quantile(0.75)
IQR = Q3 - Q1          
outliers =df[(df[i] < (Q1 - 1.5 * IQR)) | (df[i] > (Q3 + 1.5 * IQR))]
print(i,'outliers Count:',outliers.shape[0])

outliers

In [12]:
# distribution of the income in the main dataframe
px.scatter(df,x='Income',trendline="ols")

In [13]:
# distribution of the income in the outlier dataframe
px.scatter(outliers,x='Income',trendline="ols")

<h5>Need To Delete Outliers of index 3896, 4993 Form Income Column</h5>

<h3>4.2- Check Outliers For CCAvg Column</h3>

In [14]:
i = 'CCAvg'

Q1 = df[i].quantile(0.25)
Q3 = df[i].quantile(0.75)
IQR = Q3 - Q1          
outliers =df[(df[i] < (Q1 - 1.5 * IQR)) | (df[i] > (Q3 + 1.5 * IQR))]
print(i,'outliers Count:',outliers.shape[0])

outliers

In [15]:
# distribution of the CCAvg in the main dataframe
px.scatter(df,x='CCAvg',trendline="ols")

In [16]:
# distribution of the CCAvg in the outliers dataframe
px.scatter(outliers,x='CCAvg',trendline="ols")

<h5>No Need To Delete Outliers Form CCAvg Column</h5>

<h3>4.3- Check Outliers For Mortgage Column</h3>

In [17]:
i='Mortgage'

Q1 = df[i].quantile(0.25)
Q3 = df[i].quantile(0.75)
IQR = Q3 - Q1          
outliers =df[(df[i] < (Q1 - 1.5 * IQR)) | (df[i] > (Q3 + 1.5 * IQR))]
print(i,'outliers Count:',outliers.shape[0])

outliers

In [18]:
# distribution of the Mortgage in the main dataframe
px.scatter(df,x='Mortgage',trendline="ols")

In [19]:
# distribution of the Mortgage in the outliers dataframe
px.scatter(outliers,x='Mortgage',trendline="ols")

<h5>No Need To Delete Outliers Form Mortgage Column</h5>

<a id='dc'></a>
# Data Cleaning

<img src="https://media2.giphy.com/media/eMbKluW9vLGtFKI5JR/giphy.gif?cid=790b76115a267806e51ce17aa44eab93764caf217ee82620&rid=giphy.gif&ct=g
"/>

<h6>1- Convert All The Negative Values To Positive At Experience Column</h6>

In [20]:
df['Experience']=df['Experience'].apply(lambda x :abs(x))

<h6>2- Drop Unused columns</h6>

In [21]:
df.drop(['ID','ZIP Code'],axis=1,inplace=True)

In [22]:
df.drop(labels=[3896, 4993],axis=0,inplace=True)

<h6>2-Delete The Outliers</h6>

<a id='dfl'></a>
# Data Final Look

In [23]:
df.head()

In [24]:
df.info()

In [25]:
df.describe().transpose()

In [26]:
df.isnull().sum()

<a id=eda></a>
# Exploratory Data Analysis

<img src="https://media2.giphy.com/media/y31rRE5h3wyPXey8vx/giphy.gif?cid=ecf05e47tyfles51nr6kpluybtefs5d01lwgaqf53oiprkyt&rid=giphy.gif&ct=g">

## Co-relation Martex View

In [27]:
# See the power of relation between coleumns

corrMatrix = df.corr()
sn.heatmap(corrMatrix, annot=True)
plt.show()

### From the matrix, we can see some columns affect the personal loan like:
- Income " 0.5 "
- CCAvg " 0.37 "
- CD Account " 0.32 "

In [28]:
%%HTML
<div class='tableauPlaceholder' id='viz1659791234848' style='position: relative'><noscript><a href='#'><img alt='How Does Income Affect  Personal Loan ' src='https:&#47;&#47;public.tableau.com&#47;static&#47;images&#47;in&#47;incloan&#47;v1&#47;1_rss.png' style='border: none' /></a></noscript><object class='tableauViz'  style='display:none;'><param name='host_url' value='https%3A%2F%2Fpublic.tableau.com%2F' /> <param name='embed_code_version' value='3' /> <param name='site_root' value='' /><param name='name' value='incloan&#47;v1' /><param name='tabs' value='no' /><param name='toolbar' value='yes' /><param name='static_image' value='https:&#47;&#47;public.tableau.com&#47;static&#47;images&#47;in&#47;incloan&#47;v1&#47;1.png' /> <param name='animate_transition' value='yes' /><param name='display_static_image' value='yes' /><param name='display_spinner' value='yes' /><param name='display_overlay' value='yes' /><param name='display_count' value='yes' /><param name='language' value='en-US' /><param name='filter' value='publish=yes' /></object></div>                <script type='text/javascript'>                    var divElement = document.getElementById('viz1659791234848');                    var vizElement = divElement.getElementsByTagName('object')[0];                    vizElement.style.width='100%';vizElement.style.height=(divElement.offsetWidth*0.75)+'px';                    var scriptElement = document.createElement('script');                    scriptElement.src = 'https://public.tableau.com/javascripts/api/viz_v1.js';                    vizElement.parentNode.insertBefore(scriptElement, vizElement);                </script>

### Here is showen that the income has an effect on if the person will take loan or not

<br>

In [29]:
%%HTML
<div class='tableauPlaceholder' id='viz1659790810964' style='position: relative'><noscript><a href='#'><img alt='How Does Avg Spend on Credit Affect The Personal Loan ' src='https:&#47;&#47;public.tableau.com&#47;static&#47;images&#47;cc&#47;ccavgperloan&#47;v2&#47;1_rss.png' style='border: none' /></a></noscript><object class='tableauViz'  style='display:none;'><param name='host_url' value='https%3A%2F%2Fpublic.tableau.com%2F' /> <param name='embed_code_version' value='3' /> <param name='site_root' value='' /><param name='name' value='ccavgperloan&#47;v2' /><param name='tabs' value='no' /><param name='toolbar' value='yes' /><param name='static_image' value='https:&#47;&#47;public.tableau.com&#47;static&#47;images&#47;cc&#47;ccavgperloan&#47;v2&#47;1.png' /> <param name='animate_transition' value='yes' /><param name='display_static_image' value='yes' /><param name='display_spinner' value='yes' /><param name='display_overlay' value='yes' /><param name='display_count' value='yes' /><param name='language' value='en-US' /><param name='filter' value='publish=yes' /></object></div>                <script type='text/javascript'>                    var divElement = document.getElementById('viz1659790810964');                    var vizElement = divElement.getElementsByTagName('object')[0];                    vizElement.style.width='100%';vizElement.style.height=(divElement.offsetWidth*0.75)+'px';                    var scriptElement = document.createElement('script');                    scriptElement.src = 'https://public.tableau.com/javascripts/api/viz_v1.js';vizElement.parentNode.insertBefore(scriptElement, vizElement);</script>

### Here is shown that the Avg Spend on Credit Effect has not much effect on if the person will take loan or not

<br>

In [30]:
CD_loan = df.groupby('CD Account',as_index=False)['Personal Loan'].mean()
px.pie(CD_loan,names=['Don\'t Have','Have'],values='Personal Loan',title='The Affect of certificates of deposit')

### Here is shown that the people who has certificates of deposit takes loan more than the people who haven't

<br>

In [31]:
Edu_inc= df.groupby('Education',as_index=False)['Income'].mean()
px.pie(Edu_inc,names=['Un-Degree','Graduated','Professional'],values='Income',title='Education effect on Income')

### The Education level doesn't affect too much at the income, but seems like the undegree people has more income

<br>

In [32]:
CD_loan = df.groupby('Education',as_index=False)['Personal Loan'].mean()
px.pie(CD_loan,names=['Un-Degree','Graduated','Professional'],values='Personal Loan',title='The Affect of certificates of deposit')

### The Education level doesn't affect too much at the Chance to have personal loan, but seems like the professional people has more income

<br>

In [33]:
%%HTML
<div class='tableauPlaceholder' id='viz1659792125158' style='position: relative'><noscript><a href='#'><img alt='The Proportion of Taking Loan For Each Age and Education Level ' src='https:&#47;&#47;public.tableau.com&#47;static&#47;images&#47;tp&#47;tpotlfeaael&#47;v6&#47;1_rss.png' style='border: none' /></a></noscript><object class='tableauViz'  style='display:none;'><param name='host_url' value='https%3A%2F%2Fpublic.tableau.com%2F' /> <param name='embed_code_version' value='3' /> <param name='site_root' value='' /><param name='name' value='tpotlfeaael&#47;v6' /><param name='tabs' value='no' /><param name='toolbar' value='yes' /><param name='static_image' value='https:&#47;&#47;public.tableau.com&#47;static&#47;images&#47;tp&#47;tpotlfeaael&#47;v6&#47;1.png' /> <param name='animate_transition' value='yes' /><param name='display_static_image' value='yes' /><param name='display_spinner' value='yes' /><param name='display_overlay' value='yes' /><param name='display_count' value='yes' /><param name='language' value='en-US' /><param name='filter' value='publish=yes' /></object></div>                <script type='text/javascript'>                    var divElement = document.getElementById('viz1659792125158');                    var vizElement = divElement.getElementsByTagName('object')[0];                    vizElement.style.width='100%';vizElement.style.height=(divElement.offsetWidth*0.75)+'px';                    var scriptElement = document.createElement('script');                    scriptElement.src = 'https://public.tableau.com/javascripts/api/viz_v1.js';                    vizElement.parentNode.insertBefore(scriptElement, vizElement);                </script>

### Here is Shown that:
1- Ages of 25, [32 to 36], [63 to 65] have more chance to get loan<br>
2- professional degree owners are more likely to have Personal Loan<br>
3- Family Size Doesn't affect too much but the family size 3 is more likely to have Personal Loan

<br>

<a id='ml'></a>
# Random Forest Classifier Model

<img src="https://media2.giphy.com/media/NTMxntb8rMzmbd1x97/giphy.gif?cid=ecf05e4701r6tftfw2s49q9fgtiv8w5a8c0hquuye9ssvz2a&rid=giphy.gif&ct=g
"/>

In [34]:
# Split The Data

X = df.drop('Personal Loan', axis=1)
y = df['Personal Loan']

X_train, X_test, y_train, y_test = train_test_split(
    X, y, 
    test_size=0.2, random_state=42
)

In [35]:
# Fit The Model

reg_rf = RandomForestClassifier()
reg_rf.fit(X_train, y_train)

## Making The Prediction

In [36]:
y_pred = reg_rf.predict(X_test)
print(metrics.classification_report(y_test, y_pred))

<br>

 <a id='sum'></a>
 # Summary

**1-** we can see some columns affect the personal loan like:
* Income " 0.5 "
* CCAvg " 0.37 "
* CD Account " 0.32 "
<br>

**2-** The income has an effect on if the person will take a loan or not, so the higher income the higher chance to have loan
<br>

**3-** The Average Spend on Credit effect has not much effect on if the person will take a loan or not but seems that if the person has higher average may has higher chance to have a loan
<br>

**4-** The people who has certificates of deposit takes loan more than the people who haven't
<br>

**5-** The Education level doesn't affect too much at the Chance to have personal loan, but seems like the professional people has more income
<br>

**6-** Ages of 25, [32 to 36], [63 to 65] have more chance to get loan
<br>

**7-** professional degree owners are more likely to have Personal Loan
<br>

**8-** Family Size Doesn't affect too much but the family size 3 is more likely to have Personal Loan

 <a id='con'></a>
 # Conclusion

We Should to target the client with:<br>
1- higher income<br>
2- higher Average Spend on Credit<br>
3- certificates of deposit<br>
4- professional education level<br>
5- Ages of 25, [32 to 36], [63 to 65]<br>
6- Family Size of 3<br>
to have the highest chance of taking personal loan<br>

<img style="display:inline; " src="https://media4.giphy.com/media/lQ6KHKqQaPrx7CBhPD/giphy.gif?cid=790b7611f9ec670f52232fd75b6b5273e13f44c70a07c032&rid=giphy.gif&ct=g" width="300" height="400">

<img style="display:inline" src="https://media2.giphy.com/media/BYoRqTmcgzHcL9TCy1/giphy.gif?cid=790b76117545874ff11cb1e1099fbc05dcbd5567b77a1735&rid=giphy.gif&ct=g"/>

<img style="display:inline; " src="https://media4.giphy.com/media/lQ6KHKqQaPrx7CBhPD/giphy.gif?cid=790b7611f9ec670f52232fd75b6b5273e13f44c70a07c032&rid=giphy.gif&ct=g" width="300" height="400">
